In [39]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [40]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')


In [41]:
df.head()

,id,headline,url,publisher,date,stock,headline_cleaned,senti_pos_score,senti_neg_score,senti_score,open,close
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",0.125,0.000,0.125,92.13,90.38
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",0.125,0.000,0.125,90.65,90.49
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",0.250,0.125,0.125,86.23,86.13
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",0.000,0.000,0.000,85.00,84.98
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,"['security.n.01', 'neutral.n.01', 'technology....",0.500,0.250,0.250,85.00,84.98


In [42]:
def stockDevelopement(row):
    stock_differenz = row['open'] - row['close']
    return 1 - row['close']/row['open']
    if quotient < 1:
        return -1
    elif quotient > 1:
        return 1
    else: return 0

In [43]:
df['stockPriceChange'] = df.apply(stockDevelopement, axis=1)

In [44]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPriceChange"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPrice_Change':'_stockPrice_Change'})

In [45]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [46]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [47]:
df = df.set_index('id')
df.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,0.018995,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,0.001765,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,0.001160,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,0.000235,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,0.000235,security neutral technology raise monetary_val...


In [48]:
df['stockPriceChange'] = df['stockPriceChange'].apply(lambda value: value*100)
df['stockPriceChange'] = df['stockPriceChange'].astype(np.float16)
df['_open'] = df['_open'].astype(np.float16)
df['_close'] = df['_close'].astype(np.float16)
df.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.125,90.375,1.899414,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.625,90.500,0.176514,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.250,86.125,0.115967,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.000,85.000,0.023529,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.000,85.000,0.023529,security neutral technology raise monetary_val...


In [49]:
df_set = df['_headline_sentence'].tolist()

In [50]:
df.shape

(161478, 7)

In [51]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [52]:
count_wm = count_vectorizer.fit_transform(df_set)
tfidf_wm = tfidf_vectorizer.fit_transform(df_set)


In [53]:
count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

In [54]:
df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=df.index, columns=tfidf_tokenz)

In [55]:
df_tfidfvect.head()
# df_countvect.sample(50)

,131,aare,aaron,abacus,abandon,abattoir,abbot,abbreviated,abdominal,abdominal_aortic_aneurysm,...,zebra,zebu,zero,zeus,zhou,ziegler,zinc,zion,zip_up,zone
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
cols_to_delete = []
for column in df_tfidfvect:
    
    # print(df_tfidfvect[df_tfidfvect[column] > 0].shape[0])
    if df_tfidfvect[df_tfidfvect[column] > 0].shape[0] <50:
        cols_to_delete.append(column)


In [60]:
print(len(cols_to_delete))
print(df_tfidfvect.shape[1])


7095
8713


In [61]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)



In [62]:
df_tfidfvect.head()


,aaron,ability,acadia,accelerate,ache,acme,acquisition,act,acting,action,...,worst,worth,wuhan,xerox,yak,year,yesterday,yip,york,zion
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
df.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.125,90.375,1.899414,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.625,90.500,0.176514,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.250,86.125,0.115967,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.000,85.000,0.023529,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.000,85.000,0.023529,security neutral technology raise monetary_val...


In [64]:
# merged_df = pd.concat([df.set_index('id'), df_tfidfvect.set_index('id')], axis=1, join='inner')
mergred = df.merge(df_tfidfvect, on='id')

In [65]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence,aaron,ability,acadia,...,worst,worth,wuhan,xerox,yak,year,yesterday,yip,york,zion
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.125,90.375,1.899414,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.625,90.500,0.176514,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.250,86.125,0.115967,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.000,85.000,0.023529,stock travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.000,85.000,0.023529,security neutral technology raise monetary_val...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
import numpy as np

In [75]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 100t, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [30, 33, 37, 41, 44, 48, 52, 55, 59, 63, 66, 70, 74, 77, 81, 85, 88, 92, 96, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, None], 'min_samples_split': [2, 3, 5], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [76]:
from sklearn.ensemble import RandomForestRegressor


In [77]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv = 2, verbose=2, random_state=42, n_jobs = -1)

In [78]:
X = mergred.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', 'stockPriceChange', '_headline_sentence'])
y= mergred['stockPriceChange']

In [79]:
# Fit the random search model
rf_random.fit(X, y)

Fitting 2 folds for each of 25 candidates, totalling 50 fits


MemoryError: Unable to allocate 997. MiB for an array with shape (1618, 80739) and data type float64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

In [37]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=32, random_state=0,)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [36]:
from sklearn import metrics
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 3.8365657864785034
Mean Squared Error: 67.47029058351657
Root Mean Squared Error: 8.214030106075615
